In [12]:
import pandas as pd
import keras
from datetime import timedelta
from datetime import datetime
import calendar

In [13]:
data = pd.read_csv("completed_form_csv.csv")

In [14]:
data = data.iloc[:,:9]

In [15]:
data = data.loc[~data.isna().all(axis = 1)]

In [16]:
data

,PrimaryLabel,SecondaryLabel,DescriptiveLabel,START DATE,StartTime,EndTime,Enjoyment Score,Caffeine intake,Energy level
0,Sleep,NaN,NaN,17/02/2022,23:45,09:00,2.0,0.0,tired
1,Using Devices,NaN,NaN,18/02/2022,09:00,09:30,2.0,0.0,tired
2,Eating / Drinking,Using Devices,NaN,18/02/2022,09:30,10:00,2.0,1.0,calm
3,Paid Work,NaN,NaN,18/02/2022,10:00,13:00,0.0,1.0,calm
4,Travelling,Using Devices,NaN,18/02/2022,13:00,13:30,-2.0,0.0,tired
5,Education / Lectures,NaN,NaN,18/02/2022,13:30,14:30,0.0,0.0,tired
6,Eating / Drinking,Leisure,NaN,18/02/2022,14:30,15:30,2.0,1.0,calm
7,Exercise,Using Devices,NaN,18/02/2022,15:30,17:00,2.0,0.0,energetic
8,Travelling,Using Devices,NaN,18/02/2022,17:00,17:15,-1.0,0.0,calm
9,Other,Using Devices,Haircut,18/02/2022,17:15,18:30,1.0,1.0,energetic


In [17]:
primary_codes = {
'Eating / Drinking' : 'ED145',
'Education / Lectures' : 'EL642',
'Exercise' : 'X893',
'Housework' : 'H179',
'Leisure' : 'L418',
'Reading' : 'R523',
'Sleep' : 'S801',
'Travelling':'T695',
'Using Devices':'UD415',
'Paid Work' : 'PW101',
'Coursework' : 'CW982',
'Other' : 'O733'
}



In [19]:
for row in data.index:
    ## duration
    label,SecondaryLabel, date, start, end = data.loc[row,['PrimaryLabel','SecondaryLabel','START DATE','StartTime','EndTime']].copy()
    start = pd.to_datetime(' '.join([date, start]))
    end   = pd.to_datetime(' '.join([date, end]))
    
    if end < start:
        end += timedelta(days = 1)
    
    diff = pd.Timedelta(end - start).seconds / 60 
    data.loc[row,'Duration'] = diff
    
    
    ## adding in codes 
    val = primary_codes[label]
    data.loc[row,'PrimaryActivityCode'] = val
    if SecondaryLabel == SecondaryLabel:
        val2 = primary_codes[SecondaryLabel]
        data.loc[row,'SecondaryActivityCode'] = val2
    
    ## weekday
    weekday = calendar.day_name[pd.to_datetime(date).weekday()]
    data.loc[row,'Weekday'] = weekday
    
    ## bathweeknb
    nb = datetime.date(pd.to_datetime(date)).isocalendar()[1] + 13
    data.loc[row,'BathWeekNb'] = int(nb)

    

In [20]:
column_to_move = data.pop("PrimaryActivityCode")
data.insert(0, "PrimaryActivityCode", column_to_move)
column_to_move = data.pop("SecondaryActivityCode")
data.insert(1, "SecondaryActivityCode", column_to_move)
column_to_move = data.pop("Duration")
data.insert(8, "Duration", column_to_move)
column_to_move = data.pop("BathWeekNb")
data.insert(9, "BathWeekNb", column_to_move)
column_to_move = data.pop("Weekday")
data.insert(9, "Weekday", column_to_move)

In [21]:
data

,PrimaryActivityCode,SecondaryActivityCode,PrimaryLabel,SecondaryLabel,DescriptiveLabel,START DATE,StartTime,EndTime,Duration,Weekday,BathWeekNb,Enjoyment Score,Caffeine intake,Energy level
0,S801,NaN,Sleep,NaN,NaN,17/02/2022,23:45,09:00,555.0,Thursday,20.0,2.0,0.0,tired
1,UD415,NaN,Using Devices,NaN,NaN,18/02/2022,09:00,09:30,30.0,Friday,20.0,2.0,0.0,tired
2,ED145,UD415,Eating / Drinking,Using Devices,NaN,18/02/2022,09:30,10:00,30.0,Friday,20.0,2.0,1.0,calm
3,PW101,NaN,Paid Work,NaN,NaN,18/02/2022,10:00,13:00,180.0,Friday,20.0,0.0,1.0,calm
4,T695,UD415,Travelling,Using Devices,NaN,18/02/2022,13:00,13:30,30.0,Friday,20.0,-2.0,0.0,tired
5,EL642,NaN,Education / Lectures,NaN,NaN,18/02/2022,13:30,14:30,60.0,Friday,20.0,0.0,0.0,tired
6,ED145,L418,Eating / Drinking,Leisure,NaN,18/02/2022,14:30,15:30,60.0,Friday,20.0,2.0,1.0,calm
7,X893,UD415,Exercise,Using Devices,NaN,18/02/2022,15:30,17:00,90.0,Friday,20.0,2.0,0.0,energetic
8,T695,UD415,Travelling,Using Devices,NaN,18/02/2022,17:00,17:15,15.0,Friday,20.0,-1.0,0.0,calm
9,O733,UD415,Other,Using Devices,Haircut,18/02/2022,17:15,18:30,75.0,Friday,20.0,1.0,1.0,energetic


In [22]:
## export to csv
data.to_csv("completed_diary.csv")